In [ ]:
from langchain.tools import Tool
from tespy.networks import Network
from tespy.components import Compressor, Condenser, Pump, CycleCloser
from tespy.connections import Connection

# Define a TESPy function for heat pump modeling
def simulate_heat_pump(sink_temp=60):
    nw = Network(fluids=['water', 'R134a'])
    cc = CycleCloser('CycleCloser')
    comp = Compressor('Compressor')
    cond = Condenser('Condenser')
    
    # Define connections and system setup...
    
    nw.solve('design')
    cop = comp.P.val / cond.Q.val
    return f"Optimized heat pump COP: {cop:.2f}"

# Define as a LangChain tool
tespy_tool = Tool(
    name="TESPy Heat Pump Optimizer",
    func=simulate_heat_pump,
    description="Runs a TESPy simulation for heat pump optimization."
)

# Now integrate this tool into a LangChain agent
from langchain.chat_models import ChatOpenAI
from langchain.agents import initialize_agent

llm = ChatOpenAI(model="gpt-4-turbo")
agent = initialize_agent([tespy_tool], llm, agent="zero-shot-react-description")

# Example query
response = agent.run("Optimize my heat pump for maximum COP at 60°C sink temperature.")
print(response)


In [ ]:
from langchain.schema import SystemMessage, HumanMessage, AIMessage
from langchain.chains import SimpleSequentialChain
from langchain.prompts import PromptTemplate

# Define prompt for running TESPy
prompt_tespy = PromptTemplate(
    input_variables=["temperature"],
    template="Run a TESPy simulation to optimize a heat pump with a sink temperature of {temperature}°C."
)

# Define function to execute TESPy simulation
def run_tespy_simulation(temperature):
    return simulate_heat_pump(sink_temp=temperature)

# Create the chain
tespy_chain = SimpleSequentialChain(
    prompt=prompt_tespy, llm=llm, verbose=True
)

# Run the chain
response = tespy_chain.run("60")
print(response)
